# About
This notebook gives an example of how the ``signac`` framework can be used to manage a data space and automate operations on this data space.
In this example, we let's imagine that we're studying the behavior of a rocket launched at $6$ mph at an angle to visualize the distance it will travel before it lands.
We use simple Newtonian mechanics to model the motion, first with a force balance in y to find how long the object travels: 

$
\begin{equation}
    \begin{aligned}
        y(t) &= y(0) + v\sin(\theta) t - \frac{1}{2} g t^2 \\
    \end{aligned}
\end{equation}
$

Note that $y(0)=0$ and solve for $t$ such that $y(t) = 0$:

$
\begin{equation}
    \begin{aligned}
        0 &= 0 + v\sin(\theta)t - \frac{1}{2} gt^2 \\
        t &= \frac{-v \sin(\theta) \pm \sqrt{v^2 sin^2(\theta)}}{-g} \\
    \end{aligned}
\end{equation}
$

One of these solutions is evidently zero, so the root we want is given by:

$
\begin{equation}
    \begin{aligned}
        t &= \frac{2v \sin(\theta)}{g} \\
    \end{aligned}
\end{equation}
$

# Initial experiments

Our first goal is to use the simple math we've derived above to see how far the rocket travels for different launch angles.
To do this, we define some functions and test them.

In [ ]:
!rm -rf workspace signac.rc project.py *.err.* *.out.* signac_project_document.json animation.gif view

In [ ]:
import numpy as np
def get_tmax(v, theta, g = 9.81):
    return 2*v*np.sin(theta)/g

def compute_x(t, v, theta, g = 9.81):
    return v*np.cos(theta)*t

def compute_y(t, v, theta, g = 9.81):
    return v*np.sin(theta)*t - (g/2)*t**2

In [ ]:
print(get_tmax(6, 0.1))
print(get_tmax(6, 0.2))
print(get_tmax(6, 0.3))

for theta in np.arange(0.1, 0.4, 0.1):
    print("The distance traveled when \u03B8={:1.1f} is {:3.2e}".format(
        theta, compute_x(get_tmax(6, theta), 0.6, theta)))

# Initialize data space

Now, let's see how we can store this data with ``signac``. 

In [ ]:
import signac
theta = 0.4

project = signac.init_project("Projectile")
job = project.open_job({"theta": theta})
job.init()

job.doc['t_max'] = get_tmax(6, theta)
job.doc['x_max'] = compute_x(job.doc.t_max, 6, theta)
job.doc['y_max'] = compute_y(job.doc.t_max/2, 6, theta)

In [ ]:
print(job.sp())
print(job.doc())

In [ ]:
!find . -not -path '*/\.*'

# Computing the trajectory

In order to get a better idea of the behavior, we decide we want to compute and visualize the full trajectory.
To do so, let's use the data stored in ``signac``.

In [ ]:
num_samples=1000
t = np.linspace(0, job.doc['t_max'], num_samples)
x = compute_x(t, 6, job.sp.theta)
y = compute_y(t, 6, job.sp.theta)

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import animation, patches
from IPython.display import Image

num_frames = 20
interval = num_samples//num_frames

fig, ax = plt.subplots(1, 1);

def draw_frame(fn):
    ax.clear();
    ax.plot(x[:fn*interval], y[:fn*interval])
    ax.set_xlim([0, np.max(x)*1.2])
    ax.set_ylim([0, np.max(y)*1.2])
    patch = patches.Ellipse(
                xy=(x[fn*interval], y[fn*interval]),
                width=np.max(x)/14,
                height=np.max(y)/10,
                fc='#00274c')
    ax.add_patch(patch)
    return []

anim = animation.FuncAnimation(fig, draw_frame, frames=num_frames, blit=True)
plt.close()
anim.save('animation.gif', writer='imagemagick', fps=10)
Image('animation.gif')

# Expand data space

We've shown how this works for one data point.
Now we get to the real goals of ``signac``, which is interacting with larger data spaces.
Let's add more data points to our space.

In [ ]:
thetas = [0.4, 0.7, 1.0, 1.3]
for theta in thetas:
    job = project.open_job({"theta": theta})
    job.init()

    job.doc['t_max'] = get_tmax(6, theta)
    job.doc['x_max'] = compute_x(job.doc.t_max, 6, theta)
    job.doc['y_max'] = compute_y(job.doc.t_max/2, 6, theta)

## Accessing this data

Here we see the simplest method of accessing data in a project: iterating over the entire project to compute some quantity.

In [ ]:
x_max = 0
theta_max = None
for job in project:
    if job.doc.x_max > x_max:
        x_max = job.doc.x_max
        theta_max = job.sp.theta
print("The furthest distance traveled was {:4.2f} for a theta value of {}".format(x_max, theta_max))

# Changing the schema

Now imagine that we suddenly discover new fuels for our rocket that allow it to travel much faster than it originally did.
This means that we now have to account for a range of velocities in our data schema.
To do so, we can easily modify existing jobs and then

In [ ]:
v = 6
for job in project:
    job.sp.v = v

In [ ]:
vs = [6, 8, 10]
thetas = [0.4, 0.7, 1.0, 1.3]
for v in vs:
    for theta in thetas:
        sp = {"v": v, "theta": theta} 
        job = project.open_job(sp)
        job.init() # Operation is idempotent, won't affect preexisting jobs.

## Encoding the workflow

Since we're now working with a larger data space, it would be useful to see how we can automate this.
Everything that we've done so far can be encapsulated in a single Python file that uses signac-flow encode the full logic.

In [ ]:
%%writefile project.py
from matplotlib import pyplot as plt
from matplotlib import animation, patches
import numpy as np

def get_tmax(v, theta, g = 9.81):
    return 2*v*np.sin(theta)/g

def compute_x(t, v, theta, g = 9.81):
    return v*np.cos(theta)*t

def compute_y(t, v, theta, g = 9.81):
    return v*np.sin(theta)*t - (g/2)*t**2

def make_animation(v, theta, x, y, tmax, xmax, ymax, num_samples):
    num_frames = 20
    interval = num_samples//num_frames

    fig, ax = plt.subplots(1, 1);
    
    def draw_frame(fn, v, theta, x, y, tmax, xmax, ymax):
        ax.clear();
        
        if fn*interval <= tmax:
            xpos, ypos = x[fn*interval], y[fn*interval]
            x, y = x[:fn*interval], y[:fn*interval]
        else:
            xpos, ypos = x[tmax], y[tmax]
            x, y = x[:tmax], y[:tmax]
        
        ax.plot(x, y)
        ax.set_title("v = {:2.1f}, $theta$ = {:2.2f}".format(v, theta), size = 18)
        ax.set_xlim([0, xmax*1.2])
        ax.set_ylim([0, ymax*1.2])
        patch = patches.Ellipse(
                    xy=(xpos, ypos),
                    width=xmax/14,
                    height=ymax/10,
                    fc='#00274c')
        ax.add_patch(patch)
        return []

    anim = animation.FuncAnimation(fig, draw_frame, frames=num_frames, blit=True,
                                   fargs=(v, theta, x, y, tmax, xmax, ymax))
    plt.close()
    anim.save('./animation.gif', writer='imagemagick', fps=10)

import signac
from flow import FlowProject
class Project(FlowProject):
    pass

@Project.operation
@Project.post.true('t_max')
def compute_outputs(job):
    project = signac.get_project()
    job.doc['t_max'] = get_tmax(job.sp.v, job.sp.theta)
    job.doc['x_max'] = compute_x(job.doc.t_max, job.sp.v, job.sp.theta)
    job.doc['y_max'] = compute_y(job.doc.t_max/2, job.sp.v, job.sp.theta)
    
    # Store the furthest time traveled by any point
    if job.doc.t_max > project.document.get('max_t', 0):
        project.doc.max_t = job.doc.t_max
    if job.doc.x_max > project.document.get('max_x', 0):
        project.doc.max_x = job.doc.x_max
    if job.doc.y_max > project.document.get('max_y', 0):
        project.doc.max_y = job.doc.y_max
    
@Project.operation
@Project.pre.after(compute_outputs)
@Project.post.isfile('data.npz')
@Project.post.isfile('animation.gif')
def generate_trajectory(job):
    project = signac.get_project()
    num_samples = 1000
    
    t = np.linspace(0, project.doc.max_t, num_samples)
    x = compute_x(t, job.sp.v, job.sp.theta)
    y = compute_y(t, job.sp.v, job.sp.theta)
    with job:
        np.savez_compressed('data.npz', x=x, y=y, t=t, num_samples=num_samples)
        make_animation(job.sp.v, job.sp.theta, x, y,
                       int(job.doc.t_max/project.doc.max_t*num_samples),
                       project.doc.max_x, project.doc.max_y, num_samples)
        
if __name__ == "__main__":
    Project().main()

In [ ]:
!printf "The number of remaining operations is: %d\n\n" `python3 project.py next compute_outputs | wc -l`
!python3 project.py run -o compute_outputs -n 1 -v

In [ ]:
!python3 project.py run -o compute_outputs --progress

In [ ]:
!python3 project.py status -d

In [ ]:
!python3 project.py run --progress

In [ ]:
!python3 project.py status -d --only-incomplete-operations

# Extract outputs

Now that all this data has been generated it is very easy to access it in the context of ``signac`` and pull the relevant files. 

In [ ]:
vs = sorted(list(project.detect_schema()['v'][int]))
thetas = sorted(list(project.detect_schema()['theta'][float]))
print(project.detect_schema())

In [ ]:
from IPython.display import HTML
html_str = '<table>'
for v in vs:
    html_str += "<tr>"
    for theta in thetas:
        job = next(project.find_jobs({"v": v, "theta": theta}))
        html_str += '<td><img src="workspace/{}/animation.gif"></td>'.format(job)
    html_str += "</tr>"
html_str += '</table>'

HTML(html_str)

# Viewing the data

Now, this form of data storage may be cleaner in some ways, but now it's completely impossible to inspect the data space manually if you wanted to.
If you wanted to look through the filesystem to see things, you would have to look through each JSON file for the relevant metadata, which really isn't feasible.
To overcome this, we have views.

In [ ]:
project = signac.get_project()
project.create_linked_view()

**Views are dynamic links, so can be immediately updated when the data space changes.
As a result, they can also be easily customized by simply changing the order in which directory structures are constructed when views are created without affecting the data.**

In [ ]:
!find view/

In [ ]:
!find -L view/ | head -10

# Extra

## Submission to scheduler

It is now trivial to submit these jobs to a cluster instead of running them locally.
All we need to do is change our run command to a submit command, and the operation will run on any detected scheduler.

In [ ]:
!find . -name animation.gif | head -4 | xargs rm
!python3 project.py status --full --stack --pretty

In [ ]:
!python3 project.py submit

In [ ]:
from project import Project
Project().print_status(detailed=True, all_ops=True, unroll=False, pretty=True)